In [8]:
import cv2
import numpy as np
import torch
from tqdm import tqdm
from pathlib import Path
from PIL import Image
from matplotlib import pyplot as plt
from segment_anything import SamPredictor, sam_model_registry

%matplotlib inline

## Survey on CoCo-stuff

In [3]:
data_dir = '/sda1/Datasets/CoCo'

ann_dir = data_dir + '/annotations'
train_ann_dir = ann_dir + '/train2017'
valid_ann_dir = ann_dir + '/val2017'

img_dir = data_dir + '/images'
train_img_dir = img_dir + '/train2017'
valid_img_dir = img_dir + '/val2017'

In [4]:
print(train_ann_dir)
train_ann_img = sorted(Path(train_ann_dir).glob('*.png'))
print(len(train_ann_img))

/sda1/Datasets/CoCo/annotations/train2017
118287


In [7]:
for path in train_ann_img[:10]:
    mask = Image.open(path).convert('L')
    print(mask.size, np.unique(mask))

(640, 480) [ 50  54  55 120 142 164 255]
(640, 426) [ 24 105 123 125 168 181 255]
(640, 428) [ 63  85 118 123 141 171 255]
(640, 425) [ 23 123 255]
(481, 640) [  0  27 104 147 154 156 168 255]
(640, 478) [ 17 104 114 116 131 171 255]
(381, 500) [  0  18  63 112 123 156 162 168 255]
(640, 488) [  0  21 122 141 255]
(480, 640) [  2   7  12  84  96 102 139 141 148 156 168 171 174 180 255]
(640, 426) [  2   6   7  94  95 114 127 139 143 146 148 150 156 168 255]


## Filter DUTS dataset according to the mask area size

In [7]:
import shutil

img_dir = '/sda1/Datasets/DUTS/DUTS-TE/Std-Image'
mask_dir = '/sda1/Datasets/DUTS/DUTS-TE/Std-Mask'

img_paths = sorted(Path(img_dir).glob('*.jpg'))
mask_paths = sorted(Path(mask_dir).glob('*.png'))

for img_path, mask_path in zip(img_paths, mask_paths):
    img = cv2.imread(str(img_path))
    mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
    mask = mask / 255
    if np.sum(mask) > mask.shape[0] * mask.shape[1] * 0.25:
        shutil.copy(str(img_path), f'/sda1/Datasets/DUTS/DUTS-TE/Std-Image-30/{img_path.name}')
        shutil.copy(str(mask_path), f'/sda1/Datasets/DUTS/DUTS-TE/Std-Mask-30/{mask_path.name}')

## Draw contours with red color

In [7]:
import cv2
import numpy as np

img_path = '/sda1/Datasets/DUTS/DUTS-TE/DUTS-TE-Image/ILSVRC2012_test_00000025.jpg'
mask_path = '/sda1/Datasets/DUTS/DUTS-TE/DUTS-TE-Mask/ILSVRC2012_test_00000025.png'
img = cv2.imread(img_path)
mask = cv2.cvtColor(cv2.imread(mask_path), cv2.COLOR_BGR2GRAY)

mask = cv2.threshold(mask, 127, 1, cv2.THRESH_BINARY)[1]

masked_img = np.stack([mask, mask, mask], axis=-1) * img
cv2.imwrite('/home/chengxin/Documents/Nutstore/Publication/ObjectWatermark/figs/masked_img.png', masked_img, [cv2.IMWRITE_PNG_COMPRESSION, 0])

True

In [1]:
## Add whole Mask into the DUTS dataset

In [5]:
from pathlib import Path
from PIL import Image
import numpy as np

In [3]:
img_dir = '/sda1/Datasets/DUTS/DUTS-TR/Std-Image-30-All/'
img_paths = sorted(Path(img_dir).glob('*.jpg'))
print(len(img_paths))

100


In [4]:
for path in img_paths:
    img = Image.open(path).convert('RGB')
    img = img.resize((256, 256))
    img.save(path)

In [6]:
one_mask = np.ones((256, 256), dtype=np.uint8)
for i in range(100):
    Image.fromarray(one_mask * 255).save(f'/sda1/Datasets/DUTS/DUTS-TR/Std-Mask-30-All/im{i+1}.png')

## Divide the DUTS Dataset according to the object size 

In [5]:
import shutil

data_type = 'DUTS-TE'

img_dir = f'/sda1/Datasets/DUTS/{data_type}/Std-Image'
mask_dir = f'/sda1/Datasets/DUTS/{data_type}/Std-Mask'

img_paths = sorted(Path(img_dir).glob('*.jpg'))
mask_paths = sorted(Path(mask_dir).glob('*.png'))

for img_path, mask_path in zip(img_paths, mask_paths):
    mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
    mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)[1]
    ratio = np.mean(mask/255) * 100
    if ratio <= 10:
        shutil.copy(str(img_path), f'/sda1/Datasets/DUTS/{data_type}/Levels/0-10/Image/{img_path.name}')
        shutil.copy(str(mask_path), f'/sda1/Datasets/DUTS/{data_type}/Levels/0-10/Mask/{mask_path.name}')
    elif ratio <= 20:
        shutil.copy(str(img_path), f'/sda1/Datasets/DUTS/{data_type}/Levels/10-20/Image/{img_path.name}')
        shutil.copy(str(mask_path), f'/sda1/Datasets/DUTS/{data_type}/Levels/10-20/Mask/{mask_path.name}')
    elif ratio <= 30:
        shutil.copy(str(img_path), f'/sda1/Datasets/DUTS/{data_type}/Levels/20-30/Image/{img_path.name}')
        shutil.copy(str(mask_path), f'/sda1/Datasets/DUTS/{data_type}/Levels/20-30/Mask/{mask_path.name}')
    elif ratio <= 40:
        shutil.copy(str(img_path), f'/sda1/Datasets/DUTS/{data_type}/Levels/30-40/Image/{img_path.name}')
        shutil.copy(str(mask_path), f'/sda1/Datasets/DUTS/{data_type}/Levels/30-40/Mask/{mask_path.name}')
    elif ratio <= 50:
        shutil.copy(str(img_path), f'/sda1/Datasets/DUTS/{data_type}/Levels/40-50/Image/{img_path.name}')
        shutil.copy(str(mask_path), f'/sda1/Datasets/DUTS/{data_type}/Levels/40-50/Mask/{mask_path.name}')
    elif ratio <= 60:
        shutil.copy(str(img_path), f'/sda1/Datasets/DUTS/{data_type}/Levels/50-60/Image/{img_path.name}')
        shutil.copy(str(mask_path), f'/sda1/Datasets/DUTS/{data_type}/Levels/50-60/Mask/{mask_path.name}')
    elif ratio <= 70:
        shutil.copy(str(img_path), f'/sda1/Datasets/DUTS/{data_type}/Levels/60-70/Image/{img_path.name}')
        shutil.copy(str(mask_path), f'/sda1/Datasets/DUTS/{data_type}/Levels/60-70/Mask/{mask_path.name}')
    elif ratio <= 80:
        shutil.copy(str(img_path), f'/sda1/Datasets/DUTS/{data_type}/Levels/70-80/Image/{img_path.name}')
        shutil.copy(str(mask_path), f'/sda1/Datasets/DUTS/{data_type}/Levels/70-80/Mask/{mask_path.name}')
    elif ratio <= 90:
        shutil.copy(str(img_path), f'/sda1/Datasets/DUTS/{data_type}/Levels/80-90/Image/{img_path.name}')
        shutil.copy(str(mask_path), f'/sda1/Datasets/DUTS/{data_type}/Levels/80-90/Mask/{mask_path.name}')
    else:
        shutil.copy(str(img_path), f'/sda1/Datasets/DUTS/{data_type}/Levels/90-100/Image/{img_path.name}')
        shutil.copy(str(mask_path), f'/sda1/Datasets/DUTS/{data_type}/Levels/90-100/Mask/{mask_path.name}')

## Resize all image into 128 * 128

In [13]:
import shutil

image_dir = '/sda1/Datasets/DUTS/DUTS-TE/Std-Image-30/256'
output_dir = '/sda1/Datasets/DUTS/DUTS-TE/Std-Image-30/128'

image_paths = sorted(Path(image_dir).glob('*.jpg'))

for image_path in tqdm(image_paths):
    img = Image.open(image_path).convert('RGB')
    # img = Image.open(image_path).convert('L')
    img = img.resize((128, 128))
    # img.save(f'{output_dir}/{image_path.name}', compress_level=0)
    img.save(f'{output_dir}/{image_path.name}', quality=95)

100%|██████████| 2396/2396 [00:04<00:00, 489.21it/s]
